# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='2909',
        database='sakila',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Liste do maior para o menor.

In [10]:
db('SELECT name, SUM(amount) FROM category INNER JOIN film_category USING(category_id) INNER JOIN inventory USING(film_id) INNER JOIN rental USING(inventory_id) INNER JOIN payment USING(rental_id) GROUP BY name ORDER BY SUM(amount) DESC')

Executando query:
('Sports', Decimal('5314.21'))
('Sci-Fi', Decimal('4756.98'))
('Animation', Decimal('4656.30'))
('Drama', Decimal('4587.39'))
('Comedy', Decimal('4383.58'))
('Action', Decimal('4375.85'))
('New', Decimal('4351.62'))
('Games', Decimal('4281.33'))
('Foreign', Decimal('4270.67'))
('Family', Decimal('4226.07'))
('Documentary', Decimal('4217.52'))
('Horror', Decimal('3722.54'))
('Children', Decimal('3655.55'))
('Classics', Decimal('3639.59'))
('Travel', Decimal('3549.64'))
('Music', Decimal('3417.72'))


Liste os filmes e o numero de vezes em que foram alugados

In [11]:
db('SELECT title, COUNT(*) FROM film INNER JOIN inventory USING(film_id) INNER JOIN rental USING(inventory_id) GROUP BY title ORDER BY COUNT(*) DESC')

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('FORWARD TEMPLE', 32)
('GRIT CLOCKWORK', 32)
('JUGGLER HARDLY', 32)
('RIDGEMONT SUBMARINE', 32)
('SCALAWAG DUCK', 32)
('APACHE DIVINE', 31)
('GOODFELLAS SALUTE', 31)
('HOBBIT ALIEN', 31)
('NETWORK PEAK', 31)
('ROBBERS JOON', 31)
('RUSH GOODFELLAS', 31)
('TIMBERLAND SKY', 31)
('WIFE TURN', 31)
('ZORRO ARK', 31)
('BUTTERFLY CHOCOLAT', 30)
('CAT CONEHEADS', 30)
('DOGMA FAMILY', 30)
('ENGLISH BULWORTH', 30)
('FROST HEAD', 30)
('GRAFFITI LOVE', 30)
('HARRY IDAHO', 30)
('IDOLS SNATCHERS', 30)
('MARRIED GO', 30)
('MASSACRE USUAL', 30)
('MUSCLE BRIGHT', 30)
('PULP BEVERLY', 30)
('RUGRATS SHAKESPEARE', 30)
('SHOCK CABIN', 30)
('SUSPECTS QUILLS', 30)
('WITCHES PANIC', 30)
('BINGO TALENTED', 29)
('BOOGIE AMELIE', 29)
('CONFIDENTIAL INTERVIEW', 29)
('DEER VIRGINIAN', 29)
('ENEMY ODDS', 29)
('FAMILY SWEET', 29)
('GLEAMING JAWBREAKER', 29)
('GREATEST NORTH', 29)
('MOON BUNCH', 29)
('STORM HAPPINESS', 29)
('SWEETHEARTS SUSPECTS', 

Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme.

In [12]:
#média de numero de alugueis por filme
db('SELECT AVG(count) FROM (SELECT title, COUNT(*) AS count FROM film INNER JOIN inventory USING(film_id) INNER JOIN rental USING(inventory_id) GROUP BY title) AS t')
#db('SELECT title, COUNT(*) FROM film INNER JOIN inventory USING(film_id) INNER JOIN rental USING(inventory_id) GROUP BY title ORDER BY COUNT(*) WHERE COUNT(*) > 10')

Executando query:
(Decimal('16.7474'),)


In [ ]:
db('''
-- SUA QUERY AQUI!
''')

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [ ]:
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [ ]:
db('''
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
''')

In [ ]:
db('''
SELECT 
    * 
FROM 
    movie_count 
ORDER BY
    cnt DESC
LIMIT 30
''')

Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

Verifique agora a nossa *view*:

In [ ]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

In [ ]:
connection.rollback()

In [ ]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

### Vamos praticar

Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

Registre um aluguel do filme "COWBOY DOOM", feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

In [ ]:
db('''
-- SUA QUERY AQUI CASO NECESSITE DE MAIS!
''')

Verifique usando a view que a contagem de alugueis do filme subiu.

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

Faça o *rollback* desta alteração, para não modificar a nossa querida base de dados *sakila*.

In [ ]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [ ]:
db('''
DROP TABLE IF EXISTS long_film
''')

db('''
CREATE TEMPORARY TABLE long_film 
    SELECT
        *
    FROM
        film
    WHERE
        film.length > 180;
''')

Podemos verificar que a tabela `long_film` agora existe:

In [ ]:
db('DESCRIBE long_film')

Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [ ]:
db('SHOW TABLES')

Vamos listar o conteudo desta tabela:

In [ ]:
db('SELECT title FROM long_film')

Vamos apagar a tabela `long_film`:

In [ ]:
db('DROP TABLE long_film')

### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

db('''
-- SUA QUERY AQUI!
''')

 - Verifique a tabela.

In [ ]:
db('SELECT * FROM max_duration')

In [ ]:
db('DESCRIBE max_duration')

- Agora use a tabela temporária para construir uma consulta com as categorias e seus respectivos filmes mais longos:

In [ ]:
db('''
SELECT
    category_id, name, film_id, title, length
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN max_duration USING (category_id)
WHERE
    length = max_len
''')

- delete a tabela temporária

In [ ]:
db('''
DROP TABLE max_duration
''')

### Desafio!

- Gere uma tabela `actor_category_films` contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

db('''
-- SUA QUERY AQUI!
''')

In [ ]:
db('SELECT * FROM actor_category_films LIMIT 10')

In [ ]:
db('''
SELECT
    MIN(first_name), MIN(last_name), GROUP_CONCAT(films SEPARATOR "; ") as total_films
FROM
    actor_category_films
GROUP BY
    actor_id
''')

## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [ ]:
db('''
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
''')

Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [ ]:
db('SELECT @avg_rate, @stddev_rate')

Agora podemos selecionar os filmes caros!

In [ ]:
db('''
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate
LIMIT 10
''')

### Vamos praticar

Use variáveis temporárias para encontrar o ator que mais participou de filmes.

## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [ ]:
db('''
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
''')

Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [ ]:
db('DROP TABLE IF EXISTS temp_pop_actors')
db('''
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT first_name, last_name, actor_id FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    GROUP BY 
        actor_id
    ORDER BY 
        COUNT(film_id) DESC
    LIMIT 3
''')
db('''
SELECT * from temp_pop_actors
''')

Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [ ]:
db('''
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [ ]:
db('DROP TABLE temp_pop_actors')

### Vamos praticar

Liste os atores que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora).

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos filmes de atores populares, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [ ]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('''
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
''')

In [ ]:
db('DESCRIBE nomes_clientes')
db('SELECT * FROM nomes_clientes LIMIT 5')

In [ ]:
db('DROP TABLE IF EXISTS nomes_staff')
db('''
CREATE TEMPORARY TABLE nomes_staff 
    SELECT first_name, last_name FROM staff
''')

In [ ]:
db('DESCRIBE nomes_staff')
db('SELECT * FROM nomes_staff LIMIT 5')

In [ ]:
db('DROP TABLE IF EXISTS nomes_all')
db('''
CREATE TEMPORARY TABLE nomes_all 
    (SELECT * FROM nomes_staff)
    UNION 
    (SELECT * FROM nomes_clientes)
''')

In [ ]:
db('DESCRIBE nomes_all')
db('SELECT * FROM nomes_all LIMIT 5')

In [ ]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('DROP TABLE IF EXISTS nomes_staff')
db('DROP TABLE IF EXISTS nomes_all')

**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*.

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

Semana do "DAN HARRIS": liste os clientes que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas onde a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'

In [ ]:
db('''
-- SUA QUERY AQUI!
''')

## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [ ]:
connection.close()